In [2]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Cargamos el archivo csv
df=pd.read_csv('Mexico_City_Limpio.csv')
#Eliminamos la columna unnamed
df= df.drop(['Unnamed: 0'], axis=1)
df.head(20)

,last_scraped,source,description,neighborhood_overview,host_name,host_since,host_response_time,host_is_superhost,host_neighbourhood,host_verifications,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,9/25/2024,city scrape,"Dentro de Villa un estudio de arte con futon, ...","Santa Fe Mall, Interloma Park and the Lion Des...",Dici,6/28/2010,a few days or more,f,Unknown,"['email', 'phone', 'work_email']",...,4.80,4.80,4.80,4.80,4.70,1.0,1.0,0.0,0.0,0.00
1,9/26/2024,previous scrape,A new concept of hosting in mexico through a b...,No overview available,Fernando,8/9/2010,within an hour,t,Condesa,"['email', 'phone', 'work_email']",...,4.70,4.87,4.78,4.98,4.48,8.0,4.0,1.0,0.0,0.41
2,9/26/2024,city scrape,"Comfortably furnished, sunny, 2 bedroom apt., ...",No overview available,Nicholas,1/4/2011,within an hour,t,HipÃ³dromo,"['email', 'phone']",...,4.75,4.94,4.92,4.98,4.91,2.0,2.0,0.0,0.0,0.31
3,9/25/2024,city scrape,COYOACAN designer studio quiet & safe! well eq...,Coyoacan is a beautiful neighborhood famous fo...,Trisha,8/24/2010,within a few hours,t,CoyoacÃ¡n,"['email', 'phone', 'work_email']",...,4.96,4.96,4.98,4.96,4.92,3.0,2.0,1.0,0.0,0.83
4,9/25/2024,city scrape,Is the best ever place triple L <br />Location...,"Is located in the best area of Mexico City, Po...",Andrea,4/27/2011,within an hour,t,Polanco,"['email', 'phone']",...,5.00,5.00,4.73,4.91,4.82,3.0,3.0,0.0,0.0,0.11
5,9/26/2024,previous scrape,No description available,No overview available,Fernando,5/31/2011,Unknown,f,CoyoacÃ¡n,['phone'],...,4.80,4.80,4.80,4.80,4.70,3.0,1.0,2.0,0.0,0.00
6,9/25/2024,city scrape,Welcome to Your Home in Mexico City<br />We ha...,San Miguel Chapultepec is the best kept secret...,Francisco Carlos,7/7/2011,within an hour,t,San Miguel Chapultepec,"['email', 'phone']",...,4.84,4.92,4.90,4.75,4.90,7.0,5.0,2.0,0.0,2.23
7,9/25/2024,city scrape,"Stay in a private room 2 blocks away from WTC,...","Great location, feel comfortable and secure at...",Carlos,7/13/2011,within a few hours,t,NÃ¡poles,"['email', 'phone']",...,4.61,4.98,4.95,4.97,4.81,2.0,0.0,2.0,0.0,0.84
8,9/26/2024,previous scrape,No description available,We are 3 blocks away from Coyoacan Downtown. O...,Arlette And Fernando,3/17/2013,within a few hours,t,CoyoacÃ¡n,"['email', 'phone']",...,4.95,4.98,4.89,4.98,4.91,3.0,2.0,1.0,0.0,0.59
9,9/25/2024,city scrape,No description available,No overview available,Rosa MarÃ­a,3/3/2013,within a few hours,f,Unknown,"['email', 'phone']",...,5.00,5.00,5.00,4.96,4.74,1.0,1.0,0.0,0.0,0.17


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [5]:
#Verificar los valores sin repetirse de una columna
unico1 = np.unique(df['room_type'])
unico1

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [6]:
#Verificar los valores sin repetirse de una columna
unico2 = np.unique(df['bathrooms_text'])
unico2

array(['0 baths', '0 shared baths', '1 bath', '1 private bath',
       '1 shared bath', '1.5 baths', '1.5 shared baths', '10 baths',
       '10 shared baths', '11 baths', '13 baths', '13 shared baths',
       '14 baths', '15 baths', '16 baths', '2 baths', '2 shared baths',
       '2.5 baths', '2.5 shared baths', '20 baths', '3 baths',
       '3 shared baths', '3.5 baths', '3.5 shared baths', '4 baths',
       '4 shared baths', '4.5 baths', '4.5 shared baths', '46 baths',
       '49 baths', '5 baths', '5 shared baths', '5.5 baths', '6 baths',
       '6 shared baths', '6.5 baths', '7 baths', '7.5 baths', '8 baths',
       '8 shared baths', '8.5 baths', '9 baths', 'Half-bath',
       'Shared half-bath'], dtype=object)

In [7]:
#Convierto una variable a dicotómica
df['room_type']= df['room_type'].replace(["Hotel room", "Private room", "Shared room"], "No Entire home/apt")
df['room_type']

0           Entire home/apt
1           Entire home/apt
2           Entire home/apt
3           Entire home/apt
4           Entire home/apt
                ...        
10518    No Entire home/apt
10519    No Entire home/apt
10520       Entire home/apt
10521       Entire home/apt
10522       Entire home/apt
Name: room_type, Length: 10523, dtype: object

In [8]:
# Extraer el número de baños como flotante
df["bathrooms_num"] = df["bathrooms_text"].str.extract(r'(\d+\.?\d*)').astype(float)

# Crear la variable dicotómica de manera automática
df["bathrooms_text"] = np.where(df["bathrooms_num"] > 1, ">1 baño", "≤1 baño")

# Verificar los cambios
df["bathrooms_text"].value_counts()


bathrooms_text
≤1 baño    6924
>1 baño    3599
Name: count, dtype: int64

In [9]:
#Convierto una variable a dicotómica
df['host_is_superhost']= df['host_is_superhost'].replace(["0"], "f")
df['host_is_superhost']

0        f
1        t
2        t
3        t
4        t
        ..
10518    f
10519    f
10520    f
10521    f
10522    f
Name: host_is_superhost, Length: 10523, dtype: object

In [10]:
#Seleccion de variables dependientes
Var_Dep1 = df["host_is_superhost"].unique()
print(Var_Dep1)
Var_Dep2 = df["instant_bookable"].unique()
print(Var_Dep2)
Var_Dep3 = df["host_identity_verified"].unique()
print(Var_Dep3)
Var_Dep4 = df["room_type"].unique()
print(Var_Dep4)
Var_Dep5 = df["bathrooms_text"].unique()
print(Var_Dep5)


['f' 't']
['f' 't']
['t' 'f']
['Entire home/apt' 'No Entire home/apt']
['≤1 baño' '>1 baño']


**Primer modelo de regresion logica.**

In [11]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep= df[['price']]
Var_Dep1= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep
y= Var_Dep1

In [12]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [13]:
#Se escalan todos los datos
escalar = StandardScaler()

In [14]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [15]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [16]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [17]:
#Realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(3157,), dtype=object)

In [18]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confunsion 1:')
print(matriz)

Matriz de Confunsion 1:
[[2085    0]
 [1072    0]]


In [19]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="f")
print('Precision del modelo 1:')
print(precision)

Precision del modelo 1:
0.660437123851758


In [20]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo 1:')
print(exactitud)

Exactitud del modelo 1:
0.660437123851758


In [21]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="t")
print('Sensibilidad del modelo 1:')
print(sensibilidad)

Sensibilidad del modelo 1:
0.0


**Segundo modelo de regresion logistica.**

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [23]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep2= df[['review_scores_rating', 'review_scores_communication', 'review_scores_checkin']]
Var_Dep2= df['host_is_superhost']

#Redefinimos las variables
X= Vars_Indep2
y= Var_Dep2

In [24]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [25]:
#Realizamos una prediccion
y_pred2 = algoritmo.predict(X_test)
y_pred2

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(3157,), dtype=object)

In [26]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred2)
print('Matriz de Confunsion 2:')
print(matriz)

Matriz de Confunsion 2:
[[1805  279]
 [ 969  104]]


In [27]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred2, average="binary", pos_label="t")
print('Precision del modelo 2:')
print(precision)

Precision del modelo 2:
0.27154046997389036


In [28]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred2)
print('Exactitud del modelo 2:')
print(exactitud)

Exactitud del modelo 2:
0.6046879949318974


In [29]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred2, average="binary", pos_label="f")
print('Sensibilidad del modelo 2:')
print(sensibilidad)

Sensibilidad del modelo 2:
0.8661228406909789


**Tercer modelo de regresion logistica.**

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [31]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep3= df[['review_scores_rating', 'price', 'host_acceptance_rate']]
Var_Dep3= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep3
y= Var_Dep3

In [32]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [33]:
#Realizamos una prediccion
y_pred3 = algoritmo.predict(X_test)
y_pred3

array(['f', 'f', 't', ..., 'f', 'f', 'f'], shape=(3157,), dtype=object)

In [34]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred3)
print('Matriz de Confunsion 3:')
print(matriz)

Matriz de Confunsion 3:
[[2122   91]
 [ 826  118]]


In [35]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred3, average="binary", pos_label="t")
print('Precision del modelo 3:')
print(precision)

Precision del modelo 3:
0.5645933014354066


In [36]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred3)
print('Exactitud del modelo 3:')
print(exactitud)

Exactitud del modelo 3:
0.7095343680709535


In [37]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred3, average="binary", pos_label="f")
print('Sensibilidad del modelo 3:')
print(sensibilidad)

Sensibilidad del modelo 3:
0.9588793492995933


**Cuarto modelo de regresion logistica.**

In [38]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep4= df[['number_of_reviews', 'price']]
Var_Dep4= df['instant_bookable']

#Redefinimos las variables
X= Vars_Indep4
y= Var_Dep4

In [39]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [40]:
#Realizamos una prediccion
y_pred4 = algoritmo.predict(X_test)
y_pred4

array(['f', 'f', 'f', ..., 'f', 'f', 'f'], shape=(3157,), dtype=object)

In [41]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred4)
print('Matriz de Confunsion 4:')
print(matriz)

Matriz de Confunsion 4:
[[2184   33]
 [ 915   25]]


In [42]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred4, average="binary", pos_label="f")
print('Precision del modelo 4:')
print(precision)

Precision del modelo 4:
0.7047434656340755


In [43]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred4)
print('Exactitud del modelo 4:')
print(exactitud)

Exactitud del modelo 4:
0.6997149192271144


In [44]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred4, average="binary", pos_label="t")
print('Sensibilidad del modelo 4:')
print(sensibilidad)

Sensibilidad del modelo 4:
0.026595744680851064


**Quinto modelo de regresion logistica.**

In [45]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep5= df[['number_of_reviews', 'price', 'host_total_listings_count']]
Var_Dep5= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep5
y= Var_Dep5

In [46]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [47]:
#Realizamos una prediccion
y_pred5 = algoritmo.predict(X_test)
y_pred5

array(['t', 't', 't', ..., 't', 't', 't'], shape=(3157,), dtype=object)

In [48]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred5)
print('Matriz de Confunsion 5:')
print(matriz)

Matriz de Confunsion 5:
[[   0  214]
 [   0 2943]]


In [49]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred5, average="binary", pos_label="t")
print('Precision del modelo 5:')
print(precision)

Precision del modelo 5:
0.9322141273360786


In [50]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred5)
print('Exactitud del modelo 5:')
print(exactitud)

Exactitud del modelo 5:
0.9322141273360786


In [51]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred5, average="binary", pos_label="f")
print('Sensibilidad del modelo 5:')
print(sensibilidad)

Sensibilidad del modelo 5:
0.0


**Sexto modelo de regresion logistica.**

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [53]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep6= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep6= df['host_identity_verified']

#Redefinimos las variables
X= Vars_Indep6
y= Var_Dep6

In [54]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [55]:
#Realizamos una prediccion
y_pred6 = algoritmo.predict(X_test)
y_pred6

array(['t', 't', 't', ..., 't', 't', 't'], shape=(3157,), dtype=object)

In [56]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred6)
print('Matriz de Confunsion 6:')
print(matriz)

Matriz de Confunsion 6:
[[   0  219]
 [   0 2938]]


In [57]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred6, average="binary", pos_label="t")
print('Precision del modelo 6:')
print(precision)

Precision del modelo 6:
0.9306303452644916


In [58]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred6)
print('Exactitud del modelo 6:')
print(exactitud)

Exactitud del modelo 6:
0.9306303452644916


In [59]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred6, average="binary", pos_label="f")
print('Sensibilidad del modelo 6:')
print(sensibilidad)

Sensibilidad del modelo 6:
0.0


**Septimo modelo de regresion logistica.**

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [61]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep7= df[['accommodates', 'host_acceptance_rate', 'price']]
Var_Dep7= df['room_type']

#Redefinimos las variables
X= Vars_Indep7
y= Var_Dep7

In [62]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [63]:
#Realizamos una prediccion
y_pred7 = algoritmo.predict(X_test)
y_pred7

array(['Entire home/apt', 'Entire home/apt', 'No Entire home/apt', ...,
       'No Entire home/apt', 'Entire home/apt', 'No Entire home/apt'],
      shape=(3157,), dtype=object)

In [64]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred7)
print('Matriz de Confunsion 7:')
print(matriz)

Matriz de Confunsion 7:
[[1594  295]
 [ 338  930]]


In [65]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred7, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 7:')
print(precision)

Precision del modelo 7:
0.7591836734693878


In [66]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred7)
print('Exactitud del modelo 7:')
print(exactitud)

Exactitud del modelo 7:
0.7994931897370922


In [67]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred7, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 7:')
print(sensibilidad)

Sensibilidad del modelo 7:
0.8438327157226045


**Octavo modelo de regresion logistica.**

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [69]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep8= df[['number_of_reviews', 'bedrooms']]
Var_Dep8= df['room_type']

#Redefinimos las variables
X= Vars_Indep8
y= Var_Dep8

In [70]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [71]:
#Realizamos una prediccion
y_pred8 = algoritmo.predict(X_test)
y_pred8

array(['No Entire home/apt', 'Entire home/apt', 'Entire home/apt', ...,
       'Entire home/apt', 'No Entire home/apt', 'Entire home/apt'],
      shape=(3157,), dtype=object)

In [72]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred8)
print('Matriz de Confunsion 8:')
print(matriz)

Matriz de Confunsion 8:
[[1526  397]
 [ 446  788]]


In [73]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred8, average="binary", pos_label="No Entire home/apt")
print('Precision del modelo 8:')
print(precision)

Precision del modelo 8:
0.6649789029535865


In [74]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred8)
print('Exactitud del modelo 8:')
print(exactitud)

Exactitud del modelo 8:
0.7329743427304403


In [75]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred8, average="binary", pos_label="Entire home/apt")
print('Sensibilidad del modelo 8:')
print(sensibilidad)

Sensibilidad del modelo 8:
0.7935517420696828


**Noveno modelo de regresion logistica.**

In [76]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep9= df[['number_of_reviews', 'price', 'accommodates']]
Var_Dep9= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep9
y= Var_Dep9

In [77]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [78]:
#Realizamos una prediccion
y_pred9 = algoritmo.predict(X_test)
y_pred9

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '>1 baño', '≤1 baño',
       '≤1 baño'], shape=(3157,), dtype=object)

In [79]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred9)
print('Matriz de Confunsion 9:')
print(matriz)

Matriz de Confunsion 9:
[[ 461  644]
 [ 217 1835]]


In [80]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred9, average="binary", pos_label=">1 baño")
print('Precision del modelo 9:')
print(precision)

Precision del modelo 9:
0.6799410029498525


In [81]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred9)
print('Exactitud del modelo 9:')
print(exactitud)

Exactitud del modelo 9:
0.7272727272727273


In [82]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred9, average="binary", pos_label="≤1 baño")
print('Sensibilidad del modelo 9:')
print(sensibilidad)

Sensibilidad del modelo 9:
0.8942495126705653


**Decimo modelo de regresion logistica.**

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10523 entries, 0 to 10522
Data columns (total 51 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   last_scraped                                  10523 non-null  object 
 1   source                                        10523 non-null  object 
 2   description                                   10523 non-null  object 
 3   neighborhood_overview                         10523 non-null  object 
 4   host_name                                     10523 non-null  object 
 5   host_since                                    10523 non-null  object 
 6   host_response_time                            10523 non-null  object 
 7   host_is_superhost                             10523 non-null  object 
 8   host_neighbourhood                            10523 non-null  object 
 9   host_verifications                            10523 non-null 

In [84]:
#Declaramos las variables dependientes e independientes para la regresion Logistica.
Vars_Indep10= df[['host_acceptance_rate', 'review_scores_rating']]
Var_Dep10= df['bathrooms_text']

#Redefinimos las variables
X= Vars_Indep10
y= Var_Dep10

In [85]:
#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

#Se escalan todos los datos
escalar = StandardScaler()

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [86]:
#Realizamos una prediccion
y_pred10 = algoritmo.predict(X_test)
y_pred10

array(['≤1 baño', '≤1 baño', '≤1 baño', ..., '≤1 baño', '≤1 baño',
       '≤1 baño'], shape=(3157,), dtype=object)

In [87]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred10)
print('Matriz de Confunsion 10:')
print(matriz)

Matriz de Confunsion 10:
[[   0 1084]
 [   0 2073]]


In [88]:
#Calculo la precision del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred10, average="binary", pos_label="≤1 baño")
print('Precision del modelo 10:')
print(precision)

Precision del modelo 10:
0.6566360468799494


In [89]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred10)
print('Exactitud del modelo 10:')
print(exactitud)

Exactitud del modelo 10:
0.6566360468799494


In [90]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred10, average="binary", pos_label=">1 baño")
print('Sensibilidad del modelo 10:')
print(sensibilidad)

Sensibilidad del modelo 10:
0.0
